Para abordar el análisis estratégico de NeoCampus y responder a las preguntas planteadas, podemos crear un código en Python que simule el modelo financiero de la empresa bajo las diferentes condiciones y supuestos. A continuación, se presenta un modelo completo que resuelve las incognitas mediante el uso de codigo avanzado que se desarrolla con base en conocimientos adquiridos de Harvard y el Externado.


In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
from IPython.display import display

In [2]:
# Constantes y supuestos base
HORIZONTE = 5  # años
ROIC_OBJETIVO = 0.20  # 20%
TASA_IMPUESTOS = 0.30  # 30%

In [3]:
# Supuestos de ingresos
USUARIOS_ACTIVOS_INICIAL = 4000
CRECIMIENTO_USUARIOS_TRANSACCIONAL = 0.05  # 5% anual
TICKET_PROMEDIO_TRANSACCIONAL = 120000  # COP
PRECIO_MENSUAL_SUSCRIPCION = 35000  # COP
CONVERSION_INICIAL_SUSCRIPCION = 0.10  # 10%
CRECIMIENTO_SUSCRIPTORES = 0.06  # 6% mensual durante 3 años
CRECIMIENTO_SUSCRIPTORES_REDUCIDO = 0.02  # 2% mensual después de 3 años

In [4]:
# Supuestos de costos
COSTO_VARIABLE_TRANSACCIONAL = 8000  # COP por usuario
COSTO_VARIABLE_SUSCRIPCION = 10000  # COP por usuario
USUARIOS_POR_TUTOR = 200
COSTO_TUTOR_MENSUAL = 2500000  # COP
COSTO_INFRAESTRUCTURA_FIJO = 150000000  # COP
INCREMENTO_INFRA_SUPERIOR_20K = 1.20  # 20% más si hay más de 20,000 suscriptores
COSTO_AMPLIACION_INFRA = 80000000  # COP
MARKETING_TRANSACCIONAL = 25000000  # COP por año
MARKETING_SUSCRIPCION_INICIAL = 80000000  # COP en año 1
INCREMENTO_MARKETING_SUSCRIPCION = 0.15  # 15% anual

In [5]:
# Restricciones
CONVERSION_MAX_SUSCRIPCION = 0.35  # 35%
RETENCION_MENSUAL_MIN = 0.90  # 90%
RETENCION_MENSUAL_MAX = 0.97  # 97%
MAX_USUARIOS_POR_TUTOR = 200
CAPACIDAD_BASE_INFRA = 20000  # usuarios

In [35]:
# Función para calcular el VAN
def calcular_van(flujos, tasa_descuento=0.10):
    """Calcula el VAN de una serie de flujos de caja."""
    return npf.npv(tasa_descuento, flujos)

# Función modificada para calcular el TIR como ROIC
def calcular_tir(flujos, inversion_total, ebit_anual):
    """
    Calcula la TIR como el ROIC promedio durante el horizonte del proyecto.
    Ahora la TIR se calcula de la misma manera que el ROIC.
    """
    roic_anual = []
    for ebit in ebit_anual:
        if ebit > 0:
            roic = ebit * (1 - TASA_IMPUESTOS) / inversion_total if inversion_total > 0 else 0
            roic_anual.append(roic)
    
    # Retornar el promedio de ROIC de los años con EBIT positivo
    if roic_anual:
        return sum(roic_anual) / len(roic_anual)
    else:
        return 0  # Si no hay años con EBIT positivo

# Función para calcular el ROIC
def calcular_roic(ebit, inversion_total):
    """Calcula el ROIC como EBIT * (1-t) / inversión total."""
    return ebit * (1 - TASA_IMPUESTOS) / inversion_total if inversion_total > 0 else 0

def model_financiero(
    inversion_inicial,
    modalidad_fondeo="A",  # "A" o "B"
    conversion_inicial=CONVERSION_INICIAL_SUSCRIPCION,
    retencion_mensual=0.93,  # valor medio entre min y max
    crecimiento_suscriptores=CRECIMIENTO_SUSCRIPTORES,
    crecimiento_reducido=CRECIMIENTO_SUSCRIPTORES_REDUCIDO,
    duplicar_marketing=False,
    ano_duplicar_marketing=0,  # 0 significa no duplicar
    expansion_universitaria=False,
    abandono_transaccional=False,
    detener_inversion_suscripcion=False,
    competidor_global=False
):
    """
    Modelo financiero para NeoCampus.
    """
    # Cálculo de inversión total según modalidad de fondeo
    if modalidad_fondeo == "A":
        inversion_total = inversion_inicial
    else:  # Modalidad B
        inversion_total = inversion_inicial
        inversion_ano_0 = inversion_inicial * 0.7
        inversion_ano_2 = inversion_inicial * 0.3
    
    # Inicializar DataFrame para resultados
    resultados = pd.DataFrame(
        0,
        index=["Inversión", "Ingresos Transaccional", "Ingresos Suscripción", "Ingresos Totales", 
               "Costos Variables Transaccional", "Costos Variables Suscripción", 
               "Costos Tutores", "Costos Infraestructura", "Marketing Transaccional", 
               "Marketing Suscripción", "Gastos Totales", "EBITDA", "Depreciación", 
               "EBIT", "Impuestos", "Utilidad Neta", "Flujo de Caja", "Flujo de Caja Acumulado", 
               "Usuarios Transaccional", "Suscriptores Finales", "ROIC"],
        columns=range(HORIZONTE + 1)  # Año 0 a 5
    )
    
    # Inversión inicial
    if modalidad_fondeo == "A":
        resultados.loc["Inversión", 0] = -inversion_inicial
    else:  # Modalidad B
        resultados.loc["Inversión", 0] = -inversion_ano_0
        resultados.loc["Inversión", 2] = -inversion_ano_2
    
    # Usuarios transaccionales
    usuarios_transaccional = [USUARIOS_ACTIVOS_INICIAL]
    for ano in range(1, HORIZONTE + 1):
        # Si decidimos abandonar el modelo transaccional en año 3
        if abandono_transaccional and ano >= 3:
            usuarios_transaccional.append(0)
        else:
            usuarios_transaccional.append(usuarios_transaccional[-1] * (1 + CRECIMIENTO_USUARIOS_TRANSACCIONAL))
    
    # Conversión a suscripción limitada por el máximo
    conversion_efectiva = min(conversion_inicial, CONVERSION_MAX_SUSCRIPCION)
    
    # Suscriptores iniciales
    suscriptores_iniciales = usuarios_transaccional[0] * conversion_efectiva
    
    # Evolución de suscriptores mes a mes durante los 5 años (60 meses)
    suscriptores_mensual = [suscriptores_iniciales]
    
    # Calcular la evolución mensual de suscriptores
    churn_mensual = 1 - retencion_mensual
    
    for mes in range(1, 60):
        # Cambio en la tasa de crecimiento después de 3 años
        if mes < 36:  # Primeros 3 años (36 meses)
            tasa_crecimiento = crecimiento_suscriptores
        else:
            tasa_crecimiento = crecimiento_reducido
        
        # Si hay competidor global en año 3, ajustamos retención y crecimiento
        if competidor_global and mes >= 36:
            retencion_ajustada = max(retencion_mensual - 0.03, RETENCION_MENSUAL_MIN)
            churn_mensual = 1 - retencion_ajustada
        
        # Si detenemos inversión en suscripción en año 3, reducimos crecimiento
        if detener_inversion_suscripcion and mes >= 36:
            tasa_crecimiento = tasa_crecimiento * 0.5  # Reducción arbitraria del 50%
        
        # Nuevos suscriptores y pérdidas por churn
        nuevos_suscriptores = suscriptores_mensual[-1] * tasa_crecimiento
        perdida_por_churn = suscriptores_mensual[-1] * churn_mensual
        
        # Actualizar el número de suscriptores
        suscriptores_siguiente = suscriptores_mensual[-1] + nuevos_suscriptores - perdida_por_churn
        suscriptores_mensual.append(suscriptores_siguiente)
    
    # Suscriptores promedio por año (promedio de los meses de cada año)
    suscriptores_promedio_anual = [0]  # Año 0
    for ano in range(1, HORIZONTE + 1):
        inicio_mes = (ano - 1) * 12
        fin_mes = ano * 12
        suscriptores_promedio = sum(suscriptores_mensual[inicio_mes:fin_mes]) / 12
        suscriptores_promedio_anual.append(suscriptores_promedio)
    
    # Ingresos transaccionales
    for ano in range(1, HORIZONTE + 1):
        resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
        resultados.loc["Ingresos Transaccional", ano] = usuarios_transaccional[ano] * TICKET_PROMEDIO_TRANSACCIONAL
    
    # Ingresos por suscripción
    for ano in range(1, HORIZONTE + 1):
        resultados.loc["Suscriptores Finales", ano] = suscriptores_promedio_anual[ano]
        resultados.loc["Ingresos Suscripción", ano] = suscriptores_promedio_anual[ano] * PRECIO_MENSUAL_SUSCRIPCION * 12
        
        # Ajuste por competidor global en año 3 (reducción de precio)
        if competidor_global and ano >= 3:
            resultados.loc["Ingresos Suscripción", ano] *= 0.8  # Reducción del 20%
    
    # Ingresos totales
    resultados.loc["Ingresos Totales"] = resultados.loc["Ingresos Transaccional"] + resultados.loc["Ingresos Suscripción"]
    
    # Costos variables
    for ano in range(1, HORIZONTE + 1):
        resultados.loc["Costos Variables Transaccional", ano] = usuarios_transaccional[ano] * COSTO_VARIABLE_TRANSACCIONAL
        resultados.loc["Costos Variables Suscripción", ano] = suscriptores_promedio_anual[ano] * COSTO_VARIABLE_SUSCRIPCION
    
    # Costos de tutores (1 tutor cada 200 suscriptores)
    for ano in range(1, HORIZONTE + 1):
        num_tutores = np.ceil(suscriptores_promedio_anual[ano] / USUARIOS_POR_TUTOR)
        resultados.loc["Costos Tutores", ano] = num_tutores * COSTO_TUTOR_MENSUAL * 12
    
    # Costos de infraestructura
    for ano in range(1, HORIZONTE + 1):
        costo_base = COSTO_INFRAESTRUCTURA_FIJO
        
        # Ampliación por expansión universitaria
        if expansion_universitaria and ano >= 2:
            costo_base = costo_base * 2  # Duplicar infraestructura
        
        # Aumento si se supera capacidad
        if suscriptores_promedio_anual[ano] > CAPACIDAD_BASE_INFRA:
            costo_base = costo_base * INCREMENTO_INFRA_SUPERIOR_20K + COSTO_AMPLIACION_INFRA
        
        resultados.loc["Costos Infraestructura", ano] = costo_base
    
    # Marketing
    for ano in range(1, HORIZONTE + 1):
        # Marketing transaccional
        if not abandono_transaccional or ano < 3:
            resultados.loc["Marketing Transaccional", ano] = MARKETING_TRANSACCIONAL
        else:
            resultados.loc["Marketing Transaccional", ano] = 0
        
        # Marketing suscripción
        if ano == 1:
            resultados.loc["Marketing Suscripción", ano] = MARKETING_SUSCRIPCION_INICIAL
        else:
            resultados.loc["Marketing Suscripción", ano] = resultados.loc["Marketing Suscripción", ano-1] * (1 + INCREMENTO_MARKETING_SUSCRIPCION)
        
        # Ajuste por duplicación de marketing
        if duplicar_marketing and ano == ano_duplicar_marketing:
            resultados.loc["Marketing Suscripción", ano] *= 2
        
        # Ajuste por expansión universitaria
        if expansion_universitaria and ano >= 2:
            resultados.loc["Marketing Suscripción", ano] *= 1.5  # Aumento del 50%
        
        # Si detenemos inversión en suscripción en año 3
        if detener_inversion_suscripcion and ano >= 3:
            resultados.loc["Marketing Suscripción", ano] *= 0.5  # Reducir a la mitad
    
    # Gastos totales
    resultados.loc["Gastos Totales"] = (
        resultados.loc["Costos Variables Transaccional"] + 
        resultados.loc["Costos Variables Suscripción"] + 
        resultados.loc["Costos Tutores"] + 
        resultados.loc["Costos Infraestructura"] + 
        resultados.loc["Marketing Transaccional"] + 
        resultados.loc["Marketing Suscripción"]
    )
    
    # EBITDA
    resultados.loc["EBITDA"] = resultados.loc["Ingresos Totales"] - resultados.loc["Gastos Totales"]
    
    # Depreciación (inversión se deprecia linealmente en 3 años)
    if modalidad_fondeo == "A":
        depreciacion_anual = inversion_inicial / 3
        for ano in range(1, 4):
            resultados.loc["Depreciación", ano] = depreciacion_anual
    else:  # Modalidad B
        depreciacion_anual_1 = inversion_ano_0 / 3
        depreciacion_anual_2 = inversion_ano_2 / 3
        for ano in range(1, 4):
            resultados.loc["Depreciación", ano] = depreciacion_anual_1
        for ano in range(3, 6):  # Años 3, 4 y 5
            resultados.loc["Depreciación", ano] += depreciacion_anual_2
    
    # EBIT
    resultados.loc["EBIT"] = resultados.loc["EBITDA"] - resultados.loc["Depreciación"]
    
    # Impuestos
    resultados.loc["Impuestos"] = resultados.loc["EBIT"].apply(lambda x: x * TASA_IMPUESTOS if x > 0 else 0)
    
    # Utilidad Neta
    resultados.loc["Utilidad Neta"] = resultados.loc["EBIT"] - resultados.loc["Impuestos"]
    
    # Flujo de Caja (Simplificado: Utilidad Neta + Depreciación - Inversión)
    resultados.loc["Flujo de Caja"] = resultados.loc["Utilidad Neta"] + resultados.loc["Depreciación"] + resultados.loc["Inversión"]
    
    # Flujo de Caja Acumulado
    resultados.loc["Flujo de Caja Acumulado"] = resultados.loc["Flujo de Caja"].cumsum()
    
    # Calcular ROIC para cada año
    for ano in range(1, HORIZONTE + 1):
        if modalidad_fondeo == "A":
            inv_acumulada = inversion_inicial
        else:
            inv_acumulada = inversion_ano_0 + (inversion_ano_2 if ano >= 2 else 0)
        
        resultados.loc["ROIC", ano] = calcular_roic(resultados.loc["EBIT", ano], inv_acumulada)
    
    # Extraer flujos de caja para cálculos finales
    flujos_caja = resultados.loc["Flujo de Caja"].values
    
    # Extraer valores de EBIT para el cálculo de TIR como ROIC
    ebit_anual = resultados.loc["EBIT", 1:].values
    
    # Calcular VAN
    van = calcular_van(flujos_caja)
    
    # Calcular TIR como ROIC
    if modalidad_fondeo == "A":
        inv_total = inversion_inicial
    else:
        inv_total = inversion_ano_0 + inversion_ano_2
    
    tir = calcular_tir(flujos_caja, inv_total, ebit_anual)
    
    # Calcular ROIC promedio (últimos 3 años)
    roic_promedio = resultados.loc["ROIC", 3:].mean() if HORIZONTE >= 3 else None
    
    return {
        "resultados": resultados,
        "flujos_caja": flujos_caja,
        "van": van,
        "tir": tir,
        "roic_promedio": roic_promedio,
        "cumple_objetivo": roic_promedio >= ROIC_OBJETIVO if roic_promedio is not None else False
    }

# Función para encontrar la inversión máxima viable
def encontrar_inversion_maxima(modalidad_fondeo, paso=10000000, min_inversion=200000000, max_inversion=2000000000):
    """
    Encuentra la inversión máxima viable que cumple con el ROIC objetivo.
    """
    inversion_actual = min_inversion
    mejor_inversion = 0
    
    while inversion_actual <= max_inversion:
        resultado = model_financiero(inversion_actual, modalidad_fondeo)
        
        if resultado["cumple_objetivo"]:
            mejor_inversion = inversion_actual
            inversion_actual += paso
        else:
            break
    
    return mejor_inversion

# Aquí empieza la ejecución del código para el análisis
# Encontrar la inversión máxima viable para cada modalidad
inversion_maxima_A = encontrar_inversion_maxima("A")
inversion_maxima_B = encontrar_inversion_maxima("B")

print(f"Inversión máxima viable en modalidad A: COP {inversion_maxima_A:,}")
print(f"Inversión máxima viable en modalidad B: COP {inversion_maxima_B:,}")

# Calcular resultados detallados para la inversión máxima en cada modalidad
resultados_A = model_financiero(inversion_maxima_A, "A")
resultados_B = model_financiero(inversion_maxima_B, "B")



/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4630.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4862.025000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5105.12

Inversión máxima viable en modalidad A: COP 700,000,000
Inversión máxima viable en modalidad B: COP 600,000,000


/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4862.025000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5105.126250000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Val

In [36]:
# Comparar VAN y TIR
print("\nComparación de resultados:")
print(f"Modalidad A - VAN: COP {resultados_A['van']:,.0f}, TIR: {resultados_A['tir']*100:.2f}%, ROIC Promedio: {resultados_A['roic_promedio']*100:.2f}%")
print(f"Modalidad B - VAN: COP {resultados_B['van']:,.0f}, TIR: {resultados_B['tir']*100:.2f}%, ROIC Promedio: {resultados_B['roic_promedio']*100:.2f}%")

# La función buscar_combinacion_optima y el resto del código permanecerían igual


Comparación de resultados:
Modalidad A - VAN: COP 253,932,456, TIR: 15.18%, ROIC Promedio: 20.35%
Modalidad B - VAN: COP 352,534,762, TIR: 20.05%, ROIC Promedio: 20.37%


In [37]:
# Buscar combinación óptima de parámetros para evitar ampliación de infraestructura
def buscar_combinacion_optima():
    """
    Busca la combinación óptima de conversión inicial, retención y crecimiento que permite 
    alcanzar un ROIC >= 20% sin necesidad de ampliar infraestructura.
    """
    mejores_parametros = None
    mejor_roic = 0
    
    # Modalidad con mejor VAN
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Rangos a probar
    conversiones = [0.10, 0.15, 0.20, 0.25, 0.30, 0.35]
    retenciones = [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97]
    crecimientos = [0.02, 0.03, 0.04, 0.05, 0.06]
    
    for conversion in conversiones:
        for retencion in retenciones:
            for crecimiento in crecimientos:
                resultado = model_financiero(
                    inversion, 
                    modalidad, 
                    conversion_inicial=conversion,
                    retencion_mensual=retencion,
                    crecimiento_suscriptores=crecimiento
                )
                
                # Verificar si los suscriptores no superan la capacidad base
                if all(resultado["resultados"].loc["Suscriptores Finales"] <= CAPACIDAD_BASE_INFRA):
                    roic = resultado["roic_promedio"]
                    if roic >= ROIC_OBJETIVO and roic > mejor_roic:
                        mejor_roic = roic
                        mejores_parametros = {
                            "conversion": conversion,
                            "retencion": retencion,
                            "crecimiento": crecimiento,
                            "roic": roic,
                            "van": resultado["van"],
                            "suscriptores_finales": resultado["resultados"].loc["Suscriptores Finales", HORIZONTE]
                        }
    
    return mejores_parametros

# Ejecutar búsqueda de combinación óptima
combinacion_optima = buscar_combinacion_optima()
print("\nCombinación óptima sin ampliar infraestructura:")
print(f"Conversión inicial: {combinacion_optima['conversion']*100:.1f}%")
print(f"Retención mensual: {combinacion_optima['retencion']*100:.1f}%")
print(f"Crecimiento mensual: {combinacion_optima['crecimiento']*100:.1f}%")
print(f"ROIC resultante: {combinacion_optima['roic']*100:.2f}%")
print(f"VAN: COP {combinacion_optima['van']:,.0f}")
print(f"Suscriptores finales (año 5): {combinacion_optima['suscriptores_finales']:,.0f}")

# Evaluar la expansión universitaria
def evaluar_expansion_universitaria():
    """Evalúa el impacto de la expansión al segmento universitario en el año 2."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Sin expansión
    resultado_base = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"]
    )
    
    # Con expansión
    resultado_expansion = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"],
        expansion_universitaria=True
    )
    
    return {
        "base": {
            "van": resultado_base["van"],
            "roic": resultado_base["roic_promedio"],
            "flujos": resultado_base["resultados"].loc["Flujo de Caja"]
        },
        "expansion": {
            "van": resultado_expansion["van"],
            "roic": resultado_expansion["roic_promedio"],
            "flujos": resultado_expansion["resultados"].loc["Flujo de Caja"]
        }
    }

# Ejecutar evaluación de expansión universitaria
expansion = evaluar_expansion_universitaria()
print("\nEvaluación de expansión universitaria:")
print(f"VAN sin expansión: COP {expansion['base']['van']:,.0f}")
print(f"VAN con expansión: COP {expansion['expansion']['van']:,.0f}")
print(f"ROIC sin expansión: {expansion['base']['roic']*100:.2f}%")
print(f"ROIC con expansión: {expansion['expansion']['roic']*100:.2f}%")
print(f"Diferencia de VAN: COP {expansion['expansion']['van'] - expansion['base']['van']:,.0f}")

# Evaluar impacto del competidor global
def evaluar_competidor_global():
    """Evalúa el impacto del ingreso de un competidor global en el año 3."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Sin competidor
    resultado_base = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"]
    )
    
    # Con competidor
    resultado_competidor = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"],
        competidor_global=True
    )
    
    # Estrategia 1: Duplicar marketing en año 3
    resultado_marketing = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"],
        competidor_global=True,
        duplicar_marketing=True,
        ano_duplicar_marketing=3
    )
    
    return {
        "base": {
            "van": resultado_base["van"],
            "roic": resultado_base["roic_promedio"]
        },
        "competidor": {
            "van": resultado_competidor["van"],
            "roic": resultado_competidor["roic_promedio"]
        },
        "marketing": {
            "van": resultado_marketing["van"],
            "roic": resultado_marketing["roic_promedio"]
        }
    }

# Ejecutar evaluación del impacto del competidor global
competidor = evaluar_competidor_global()
print("\nEvaluación del impacto del competidor global (año 3):")
print(f"VAN sin competidor: COP {competidor['base']['van']:,.0f}")
print(f"VAN con competidor: COP {competidor['competidor']['van']:,.0f}")
print(f"VAN con competidor y duplicando marketing: COP {competidor['marketing']['van']:,.0f}")
print(f"ROIC sin competidor: {competidor['base']['roic']*100:.2f}%")
print(f"ROIC con competidor: {competidor['competidor']['roic']*100:.2f}%")
print(f"ROIC con competidor y duplicando marketing: {competidor['marketing']['roic']*100:.2f}%")
print(f"Impacto del competidor en VAN: COP {competidor['competidor']['van'] - competidor['base']['van']:,.0f}")
print(f"Mejora por estrategia de marketing: COP {competidor['marketing']['van'] - competidor['competidor']['van']:,.0f}")

# Evaluar opciones en año 3
def evaluar_opciones_ano_3():
    """Evalúa las dos rutas en el año 3: eliminar modelo transaccional vs frenar inversión en suscripción."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Sin cambios (base)
    resultado_base = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"]
    )
    
    # Opción 1: Eliminar modelo transaccional
    resultado_eliminar = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"],
        abandono_transaccional=True
    )
    
    # Opción 2: Frenar inversión en suscripción
    resultado_frenar = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"],
        detener_inversion_suscripcion=True
    )
    
    return {
        "base": {
            "van": resultado_base["van"],
            "roic": resultado_base["roic_promedio"]
        },
        "eliminar": {
            "van": resultado_eliminar["van"],
            "roic": resultado_eliminar["roic_promedio"]
        },
        "frenar": {
            "van": resultado_frenar["van"],
            "roic": resultado_frenar["roic_promedio"]
        }
    }

# Ejecutar evaluación de opciones en año 3
opciones_ano_3 = evaluar_opciones_ano_3()
print("\nEvaluación de opciones en año 3:")
print(f"VAN sin cambios: COP {opciones_ano_3['base']['van']:,.0f}")
print(f"VAN eliminando modelo transaccional: COP {opciones_ano_3['eliminar']['van']:,.0f}")
print(f"VAN frenando inversión en suscripción: COP {opciones_ano_3['frenar']['van']:,.0f}")
print(f"ROIC sin cambios: {opciones_ano_3['base']['roic']*100:.2f}%")
print(f"ROIC eliminando modelo transaccional: {opciones_ano_3['eliminar']['roic']*100:.2f}%")
print(f"ROIC frenando inversión en suscripción: {opciones_ano_3['frenar']['roic']*100:.2f}%")


/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4630.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4862.025000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5105.12


Combinación óptima sin ampliar infraestructura:
Conversión inicial: 35.0%
Retención mensual: 97.0%
Crecimiento mensual: 6.0%
ROIC resultante: 119.33%
VAN: COP 2,078,524,453
Suscriptores finales (año 5): 3,273

Evaluación de expansión universitaria:
VAN sin expansión: COP 2,078,524,453
VAN con expansión: COP 1,294,560,439
ROIC sin expansión: 119.33%
ROIC con expansión: 63.34%
Diferencia de VAN: COP -783,964,014

Evaluación del impacto del competidor global (año 3):
VAN sin competidor: COP 2,078,524,453
VAN con competidor: COP 1,497,571,118
VAN con competidor y duplicando marketing: COP 1,322,941,436
ROIC sin competidor: 119.33%
ROIC con competidor: 71.32%
ROIC con competidor y duplicando marketing: 57.04%
Impacto del competidor en VAN: COP -580,953,335
Mejora por estrategia de marketing: COP -174,629,683

Evaluación de opciones en año 3:
VAN sin cambios: COP 2,078,524,453
VAN eliminando modelo transaccional: COP 1,332,868,924
VAN frenando inversión en suscripción: COP 2,112,676,718
ROI

In [38]:
# Análisis de sensibilidad para variables críticas
def analisis_sensibilidad():
    """Realiza un análisis de sensibilidad para identificar las variables críticas."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    base = model_financiero(
        inversion, 
        modalidad, 
        conversion_inicial=combinacion_optima["conversion"],
        retencion_mensual=combinacion_optima["retencion"],
        crecimiento_suscriptores=combinacion_optima["crecimiento"]
    )
    
    van_base = base["van"]
    
    # Variables a analizar y rangos de variación
    variables = {
        "conversion_inicial": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35],
        "retencion_mensual": [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97],
        "crecimiento_suscriptores": [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
    }
    
    resultados = {}
    
    # Para cada variable, calcular el impacto en el VAN
    for variable, valores in variables.items():
        impactos = []
        
        for valor in valores:
            if variable == "conversion_inicial":
                resultado = model_financiero(
                    inversion, modalidad, 
                    conversion_inicial=valor,
                    retencion_mensual=combinacion_optima["retencion"],
                    crecimiento_suscriptores=combinacion_optima["crecimiento"]
                )
            elif variable == "retencion_mensual":
                resultado = model_financiero(
                    inversion, modalidad, 
                    conversion_inicial=combinacion_optima["conversion"],
                    retencion_mensual=valor,
                    crecimiento_suscriptores=combinacion_optima["crecimiento"]
                )
            elif variable == "crecimiento_suscriptores":
                resultado = model_financiero(
                    inversion, modalidad, 
                    conversion_inicial=combinacion_optima["conversion"],
                    retencion_mensual=combinacion_optima["retencion"],
                    crecimiento_suscriptores=valor
                )
            
            van = resultado["van"]
            variacion_porcentual = (van - van_base) / abs(van_base) * 100 if van_base != 0 else 0
            
            impactos.append({
                "valor": valor,
                "van": van,
                "variacion_porcentual": variacion_porcentual
            })
        
        resultados[variable] = impactos
    
    return resultados

# Ejecutar análisis de sensibilidad
sensibilidad = analisis_sensibilidad()

# Imprimir resultados
print("\nAnálisis de sensibilidad - Impacto en VAN:")
for variable, impactos in sensibilidad.items():
    print(f"\nVariable: {variable}")
    for impacto in impactos:
        print(f"  Valor: {impacto['valor']:.2f}, VAN: COP {impacto['van']:,.0f}, Variación: {impacto['variacion_porcentual']:.2f}%")

# Análisis de escenarios con competidor global y respuestas estratégicas
def analisis_escenarios_competidor():
    """Analiza diferentes escenarios con entrada del competidor global y respuestas estratégicas."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Definir escenarios
    escenarios = [
        {"nombre": "Sin competidor", "competidor": False, "duplicar_marketing": False, "ano_duplicar": 0, "expansion": False},
        {"nombre": "Competidor sin respuesta", "competidor": True, "duplicar_marketing": False, "ano_duplicar": 0, "expansion": False},
        {"nombre": "Competidor + Marketing", "competidor": True, "duplicar_marketing": True, "ano_duplicar": 3, "expansion": False},
        {"nombre": "Competidor + Expansión", "competidor": True, "duplicar_marketing": False, "ano_duplicar": 0, "expansion": True},
        {"nombre": "Competidor + Marketing + Expansión", "competidor": True, "duplicar_marketing": True, "ano_duplicar": 3, "expansion": True}
    ]
    
    resultados_escenarios = []
    
    for escenario in escenarios:
        resultado = model_financiero(
            inversion, 
            modalidad, 
            conversion_inicial=combinacion_optima["conversion"],
            retencion_mensual=combinacion_optima["retencion"],
            crecimiento_suscriptores=combinacion_optima["crecimiento"],
            competidor_global=escenario["competidor"],
            duplicar_marketing=escenario["duplicar_marketing"],
            ano_duplicar_marketing=escenario["ano_duplicar"],
            expansion_universitaria=escenario["expansion"]
        )
        
        resultados_escenarios.append({
            "nombre": escenario["nombre"],
            "van": resultado["van"],
            "tir": resultado["tir"] if resultado["tir"] is not None else 0,
            "roic": resultado["roic_promedio"],
            "flujos": resultado["resultados"].loc["Flujo de Caja"]
        })
    
    return resultados_escenarios

# Ejecutar análisis de escenarios
escenarios = analisis_escenarios_competidor()

# Imprimir resultados
print("\nAnálisis de escenarios con competidor global:")
for escenario in escenarios:
    print(f"\nEscenario: {escenario['nombre']}")
    print(f"VAN: COP {escenario['van']:,.0f}")
    print(f"TIR: {escenario['tir']*100:.2f}%")
    print(f"ROIC promedio: {escenario['roic']*100:.2f}%")

# Análisis de sensibilidad del precio de suscripción
def analisis_sensibilidad_precio():
    """Analiza el impacto de diferentes precios de suscripción en los resultados."""
    modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion = inversion_maxima_A if modalidad == "A" else inversion_maxima_B
    
    # Precios a analizar (porcentaje de variación respecto al precio base)
    variaciones = [-0.20, -0.10, -0.05, 0, 0.05, 0.10, 0.20]
    
    class CustomFinancialModel(model_financiero.__class__):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
        
        def __call__(self, *args, **kwargs):
            global PRECIO_MENSUAL_SUSCRIPCION
            precio_original = PRECIO_MENSUAL_SUSCRIPCION
            precio_ajustado = kwargs.pop('precio_ajustado', precio_original)
            PRECIO_MENSUAL_SUSCRIPCION = precio_ajustado
            resultado = super().__call__(*args, **kwargs)
            PRECIO_MENSUAL_SUSCRIPCION = precio_original
            return resultado
    
    custom_model = CustomFinancialModel()
    
    resultados_precio = []
    precio_base = PRECIO_MENSUAL_SUSCRIPCION
    
    for variacion in variaciones:
        precio_ajustado = precio_base * (1 + variacion)
        
        resultado = custom_model(
            inversion, 
            modalidad,
            conversion_inicial=combinacion_optima["conversion"],
            retencion_mensual=combinacion_optima["retencion"],
            crecimiento_suscriptores=combinacion_optima["crecimiento"],
            precio_ajustado=precio_ajustado
        )
        
        resultados_precio.append({
            "variacion": variacion,
            "precio": precio_ajustado,
            "van": resultado["van"],
            "roic": resultado["roic_promedio"]
        })
    
    return resultados_precio

# Ejecutar análisis de sensibilidad del precio
try:
    resultados_precio = analisis_sensibilidad_precio()
    
    # Imprimir resultados
    print("\nAnálisis de sensibilidad del precio de suscripción:")
    for resultado in resultados_precio:
        print(f"Variación: {resultado['variacion']*100:+.0f}%, Precio: COP {resultado['precio']:,.0f}, "
              f"VAN: COP {resultado['van']:,.0f}, ROIC: {resultado['roic']*100:.2f}%")
except Exception as e:
    print(f"Error en análisis de sensibilidad de precio: {e}")

# Resumen ejecutivo de hallazgos
def resumen_ejecutivo():
    """Genera un resumen ejecutivo de los principales hallazgos."""
    mejor_modalidad = "A" if resultados_A["van"] > resultados_B["van"] else "B"
    inversion_maxima = inversion_maxima_A if mejor_modalidad == "A" else inversion_maxima_B
    
    mejor_escenario = max(escenarios, key=lambda x: x["van"])
    peor_escenario = min(escenarios, key=lambda x: x["van"])
    
    mejor_opcion_ano3 = max([opciones_ano_3["base"], opciones_ano_3["eliminar"], opciones_ano_3["frenar"]], 
                         key=lambda x: x["van"])
    
    # Identificando variable más crítica
    max_variaciones = {}
    for variable, impactos in sensibilidad.items():
        variaciones = [abs(impacto["variacion_porcentual"]) for impacto in impactos]
        max_variaciones[variable] = max(variaciones)
    
    variable_mas_critica = max(max_variaciones, key=max_variaciones.get)
    
    print("\n========== RESUMEN EJECUTIVO ==========")
    print(f"1. Inversión máxima viable: COP {inversion_maxima:,.0f} bajo modalidad {mejor_modalidad}")
    print(f"2. Mejor combinación sin ampliar infraestructura:")
    print(f"   - Conversión inicial: {combinacion_optima['conversion']*100:.1f}%")
    print(f"   - Retención mensual: {combinacion_optima['retencion']*100:.1f}%")
    print(f"   - Crecimiento mensual: {combinacion_optima['crecimiento']*100:.1f}%")
    print(f"3. La variable más crítica para el éxito es: {variable_mas_critica}")
    print(f"4. Mejor escenario estratégico: {mejor_escenario['nombre']}")
    print(f"   VAN: COP {mejor_escenario['van']:,.0f}, ROIC: {mejor_escenario['roic']*100:.2f}%")
    print(f"5. Mejor opción en año 3: {'Eliminar modelo transaccional' if mejor_opcion_ano3 == opciones_ano_3['eliminar'] else 'Frenar inversión en suscripción' if mejor_opcion_ano3 == opciones_ano_3['frenar'] else 'Mantener ambos modelos'}")
    print(f"6. Impacto del competidor global puede reducir el VAN en: COP {abs(escenarios[0]['van'] - escenarios[1]['van']):,.0f}")
    print("=========================================")

# Ejecutar resumen ejecutivo
resumen_ejecutivo()

# Recomendaciones estratégicas basadas en el análisis
print("\n========== RECOMENDACIONES ESTRATÉGICAS ==========")

# Modalidad de fondeo
if resultados_A["van"] > resultados_B["van"]:
    print("1. Modalidad de fondeo: Se recomienda la opción A (100% fondeo inicial)")
    print(f"   - Inversión máxima viable: COP {inversion_maxima_A:,.0f}")
    print(f"   - VAN proyectado: COP {resultados_A['van']:,.0f}")
    print(f"   - TIR proyectada: {resultados_A['tir']*100:.2f}%")
else:
    print("1. Modalidad de fondeo: Se recomienda la opción B (70% inicial, 30% en año 2)")
    print(f"   - Inversión máxima viable: COP {inversion_maxima_B:,.0f}")
    print(f"   - VAN proyectado: COP {resultados_B['van']:,.0f}")
    print(f"   - TIR proyectada: {resultados_B['tir']*100:.2f}%")

# Estrategia de crecimiento
print(f"2. Estrategia de crecimiento para lograr ROIC ≥ 20% sin ampliar infraestructura:")
print(f"   - Conversión inicial objetivo: {combinacion_optima['conversion']*100:.1f}%")
print(f"   - Retención mensual objetivo: {combinacion_optima['retencion']*100:.1f}%")
print(f"   - Crecimiento mensual objetivo: {combinacion_optima['crecimiento']*100:.1f}%")
print(f"   - Suscriptores proyectados en año 5: {combinacion_optima['suscriptores_finales']:,.0f}")

# Expansión universitaria
if expansion["expansion"]["van"] > expansion["base"]["van"]:
    print("3. Expansión universitaria: SE RECOMIENDA la expansión en año 2")
    print(f"   - Incremento proyectado de VAN: COP {expansion['expansion']['van'] - expansion['base']['van']:,.0f}")
else:
    print("3. Expansión universitaria: NO se recomienda la expansión en año 2")
    print(f"   - La expansión reduciría el VAN en: COP {abs(expansion['expansion']['van'] - expansion['base']['van']):,.0f}")

# Estrategia ante competidor global
mejor_estrategia_competidor = max([competidor["base"], competidor["competidor"], competidor["marketing"]], 
                               key=lambda x: x["van"])

if mejor_estrategia_competidor == competidor["marketing"]:
    print("4. Ante competidor global: Se recomienda duplicar marketing en año 3")
elif mejor_estrategia_competidor == competidor["base"]:
    print("4. Ante competidor global: La mejor estrategia es preventiva (evitar entrada)")
else:
    print("4. Ante competidor global: No hay respuesta efectiva, mantener operación normal")

# Decisión año 3
mejor_opcion_3 = max([opciones_ano_3["base"], opciones_ano_3["eliminar"], opciones_ano_3["frenar"]], 
                  key=lambda x: x["van"])

if mejor_opcion_3 == opciones_ano_3["eliminar"]:
    print("5. Decisión año 3: Eliminar modelo transaccional")
    print(f"   - Incremento proyectado de VAN: COP {opciones_ano_3['eliminar']['van'] - opciones_ano_3['base']['van']:,.0f}")
elif mejor_opcion_3 == opciones_ano_3["frenar"]:
    print("5. Decisión año 3: Frenar inversión en suscripción")
    print(f"   - Incremento proyectado de VAN: COP {opciones_ano_3['frenar']['van'] - opciones_ano_3['base']['van']:,.0f}")
else:
    print("5. Decisión año 3: Mantener ambos modelos sin cambios")

print("=====================================================")

# Gráfico de flujo de caja para las diferentes opciones estratégicas
print("\nFlujos de caja comparativos por año:")
print("Año 0:")
print(f"  Modalidad A: COP {resultados_A['flujos_caja'][0]:,.0f}")
print(f"  Modalidad B: COP {resultados_B['flujos_caja'][0]:,.0f}")

for ano in range(1, HORIZONTE + 1):
    print(f"Año {ano}:")
    print(f"  Modalidad A: COP {resultados_A['resultados'].loc['Flujo de Caja', ano]:,.0f}")
    print(f"  Modalidad B: COP {resultados_B['resultados'].loc['Flujo de Caja', ano]:,.0f}")


Análisis de sensibilidad - Impacto en VAN:

Variable: conversion_inicial
  Valor: 0.05, VAN: COP 425,990,978, Variación: -79.51%
  Valor: 0.10, VAN: COP 703,634,476, Variación: -66.15%
  Valor: 0.15, VAN: COP 966,804,299, Variación: -53.49%
  Valor: 0.20, VAN: COP 1,261,803,170, Variación: -39.29%
  Valor: 0.25, VAN: COP 1,523,669,058, Variación: -26.69%
  Valor: 0.30, VAN: COP 1,783,525,583, Variación: -14.19%
  Valor: 0.35, VAN: COP 2,078,524,453, Variación: 0.00%

Variable: retencion_mensual
  Valor: 0.90, VAN: COP 526,720,054, Variación: -74.66%
  Valor: 0.91, VAN: COP 620,032,874, Variación: -70.17%
  Valor: 0.92, VAN: COP 704,409,678, Variación: -66.11%
  Valor: 0.93, VAN: COP 838,955,154, Variación: -59.64%
  Valor: 0.94, VAN: COP 1,022,664,396, Variación: -50.80%
  Valor: 0.95, VAN: COP 1,231,833,392, Variación: -40.74%
  Valor: 0.96, VAN: COP 1,596,305,729, Variación: -23.20%
  Valor: 0.97, VAN: COP 2,078,524,453, Variación: 0.00%

Variable: crecimiento_suscriptores
  Valor: 

/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4630.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4862.025000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc["Usuarios Transaccional", ano] = usuarios_transaccional[ano]
/var/folders/d2/49th9bdx3rsd9plyqglb10q80000gn/T/ipykernel_4423/3223893663.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5105.12